In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

1. Load the Data

In [2]:
file_path = '/Users/jingao/Desktop/info_visual_Jingao/sf_trees.csv'
df = pd.read_csv(file_path)

/var/folders/mt/n7f_7dc9793602rk6vb3_4rm0000gn/T/ipykernel_7071/2699037970.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


4. Check Unique Values in Each Column

In [3]:
unique_values = df.nunique()
print(unique_values)

tree_id         192987
legal_status         9
species            571
address          85909
site_order         311
site_info           31
caretaker           22
date              7404
dbh                109
plot_size          524
latitude        162454
longitude       162388
dtype: int64


6. Clean data: Keep only the specified columns

In [4]:
null_values = df.isnull().sum()
null_values


tree_id              0
legal_status        54
species              0
address           1487
site_order        1634
site_info            0
caretaker            0
date            124610
dbh              41819
plot_size        50013
latitude          2832
longitude         2832
dtype: int64

In [5]:
# Removing rows with any null values
df = df.dropna()
null_values = df.isnull().sum()
null_values

tree_id         0
legal_status    0
species         0
address         0
site_order      0
site_info       0
caretaker       0
date            0
dbh             0
plot_size       0
latitude        0
longitude       0
dtype: int64

In [6]:
df = df.copy()

# Splitting the 'species' column at "::" and retaining only the common name
# If there's no common name (empty string after split), replace with 'Other Tree'
df['species'] = df['species'].str.split(' :: ').str.get(1).fillna('Other Tree')
# Displaying the first few rows to verify the changes
df.head()

,tree_id,legal_status,species,address,site_order,site_info,caretaker,date,dbh,plot_size,latitude,longitude
52,30372,DPW Maintained,Chinese Elm,498x Arkansas St,1.0,Sidewalk: Curb side : Cutout,Private,1956-03-02,10.0,3x3,37.760045,-122.398320
102,30460,DPW Maintained,Victorian Box,470 Union St,1.0,Sidewalk: Curb side : Cutout,Private,1956-05-11,19.0,4x4,37.800739,-122.407264
106,30454,DPW Maintained,Victorian Box,381 Union St,1.0,Sidewalk: Curb side : Cutout,Private,1956-05-11,8.0,3x3,37.800813,-122.405701
116,30428,DPW Maintained,Victorian Box,434 Union St,1.0,Sidewalk: Curb side : Cutout,Private,1956-05-11,13.0,7x3,37.800822,-122.406603
135,30468,DPW Maintained,Cajeput,485 Union St,2.0,Sidewalk: Curb side : Cutout,Private,1956-05-29,8.0,3x3,37.800613,-122.407292


In [7]:
species_counts = df['species'].value_counts()
species_counts.head(50)

species
Brisbane Box                          2281
Swamp Myrtle                          2012
Hybrid Strawberry Tree                1338
Other Tree                            1248
Sycamore: London Plane                 962
Southern Magnolia                      918
Victorian Box                          871
New Zealand Xmas Tree                  727
Kwanzan Flowering Cherry               647
Small-leaf Tristania 'Elegant'         548
Bronze Loquat                          515
Maidenhair Tree                        511
Ornamental Cherry                      453
Olive Tree                             437
Cajeput                                411
Primrose Tree                          392
Evergreen Pear                         377
Ornamental Pear                        371
Mayten                                 328
Cherry Plum                            296
Chinese Elm                            267
Little Gem Magnolia                    256
Samuel Sommer Magnolia                 248
Yar

In [8]:
df = df[df['species'] != 'Other Tree']
df['species'] .value_counts()

species
Brisbane Box               2281
Swamp Myrtle               2012
Hybrid Strawberry Tree     1338
Sycamore: London Plane      962
Southern Magnolia           918
                           ... 
Cabbage tree                  1
Chinese scholar tree          1
European Hornbeam             1
Narrow-leaved Ash             1
Taiwan Flowering Cherry       1
Name: count, Length: 308, dtype: int64

In [9]:
df['decade'] = (df['date'].str[:3] + '0s').astype(str)
df.head()

,tree_id,legal_status,species,address,site_order,site_info,caretaker,date,dbh,plot_size,latitude,longitude,decade
52,30372,DPW Maintained,Chinese Elm,498x Arkansas St,1.0,Sidewalk: Curb side : Cutout,Private,1956-03-02,10.0,3x3,37.760045,-122.398320,1950s
102,30460,DPW Maintained,Victorian Box,470 Union St,1.0,Sidewalk: Curb side : Cutout,Private,1956-05-11,19.0,4x4,37.800739,-122.407264,1950s
106,30454,DPW Maintained,Victorian Box,381 Union St,1.0,Sidewalk: Curb side : Cutout,Private,1956-05-11,8.0,3x3,37.800813,-122.405701,1950s
116,30428,DPW Maintained,Victorian Box,434 Union St,1.0,Sidewalk: Curb side : Cutout,Private,1956-05-11,13.0,7x3,37.800822,-122.406603,1950s
135,30468,DPW Maintained,Cajeput,485 Union St,2.0,Sidewalk: Curb side : Cutout,Private,1956-05-29,8.0,3x3,37.800613,-122.407292,1950s


In [10]:
species_counts = df['species'].value_counts()
species_over_1000 = species_counts[species_counts > 1000]
species_over_1000

species
Brisbane Box              2281
Swamp Myrtle              2012
Hybrid Strawberry Tree    1338
Name: count, dtype: int64

In [11]:
# 创建包含特定树种名称的列表
selected_species = [
    "Brisbane Box", "Swamp Myrtle", "Hybrid Strawberry Tree", "Cherry Plum",
    "Southern Magnolia", "New Zealand Xmas Tree", "Victorian Box",
    "Sycamore: London Plane", "Ornamental Cherry", "Kwanzan Flowering Cherry",
    "Mayten", "Maidenhair Tree", "Bronze Loquat", "Olive Tree",
    "Small-leaf Tristania 'Elegant'", "Ornamental Pear"
]

# 筛选原始 DataFrame 以保留只包含这些树种的行
df = df[
    df['species'].isin(selected_species)
]

df

,tree_id,legal_status,species,address,site_order,site_info,caretaker,date,dbh,plot_size,latitude,longitude,decade
102,30460,DPW Maintained,Victorian Box,470 Union St,1.0,Sidewalk: Curb side : Cutout,Private,1956-05-11,19.0,4x4,37.800739,-122.407264,1950s
106,30454,DPW Maintained,Victorian Box,381 Union St,1.0,Sidewalk: Curb side : Cutout,Private,1956-05-11,8.0,3x3,37.800813,-122.405701,1950s
116,30428,DPW Maintained,Victorian Box,434 Union St,1.0,Sidewalk: Curb side : Cutout,Private,1956-05-11,13.0,7x3,37.800822,-122.406603,1950s
262,53285,DPW Maintained,Victorian Box,2641 Franklin St,1.0,Sidewalk: Curb side : Cutout,Private,1969-08-25,20.0,4x5,37.798889,-122.425801,1960s
265,53310,DPW Maintained,Victorian Box,2427 29th Ave,1.0,Sidewalk: Curb side : Cutout,Private,1969-08-28,12.0,3x3,37.742038,-122.486442,1960s
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68363,144226,DPW Maintained,Bronze Loquat,786 43rd Ave,1.0,Sidewalk: Curb side : Cutout,Private,2020-01-25,0.0,Width 4ft,37.773865,-122.503552,2020s
68365,146918,DPW Maintained,Swamp Myrtle,787 43rd Ave,1.0,Sidewalk: Curb side : Cutout,Private,2020-01-25,0.0,Width 3ft,37.773806,-122.503706,2020s
68366,144166,DPW Maintained,Swamp Myrtle,656 41st Ave,1.0,Sidewalk: Curb side : Cutout,Private,2020-01-25,0.0,Width 4ft,37.776362,-122.501584,2020s
68373,144157,DPW Maintained,Swamp Myrtle,746 41st Ave,1.0,Sidewalk: Curb side : Cutout,Private,2020-01-25,0.0,Width 4ft,37.774642,-122.501452,2020s


In [12]:
# Save the cleaned data to a new CSV file
df.to_csv('sf_trees_cleaned.csv', index=False)

6.match tree location to San Francisco area.

In [13]:
df_trees = pd.read_csv('sf_trees_cleaned.csv')
df_areas = gpd.read_file('san-francisco-ca_.json')

In [14]:
# 步骤 3: 将树木数据转换为地理数据
gdf_trees = gpd.GeoDataFrame(df_trees, geometry=gpd.points_from_xy(df_trees.longitude, df_trees.latitude))

# 确保两个 GeoDataFrame 使用相同的坐标参考系统
df_trees.crs = df_areas.crs

# 步骤 4: 空间连接 - 将每棵树匹配到其所在的地区
df_joined = gpd.sjoin(gdf_trees, df_areas, how="inner", op="within")


/Users/jingao/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/var/folders/mt/n7f_7dc9793602rk6vb3_4rm0000gn/T/ipykernel_7071/3275223291.py:8: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  df_joined = gpd.sjoin(gdf_trees, df_areas, how="inner", op="within")


In [26]:
def categorize_decade(decade):
    decade_start = int(decade[:4])  # Convert the first 4 characters to an integer
    if 1950 <= decade_start <= 1960:
        return '1950s-1960s'
    elif 1970 <= decade_start <= 1980:
        return '1970s-1980s'
    elif 1990 <= decade_start <= 2000:
        return '1990s-2000s'
    else:
        return '2010s-2020s'

# Apply the function to create a new column
df_joined['decade_group'] = df_joined['decade'].apply(categorize_decade)

# Group by the new column
gdf_joined = df_joined.groupby('decade_group')

# Display the first few rows of each group
gdf_joined.head()

,tree_id,legal_status,species,address,site_order,site_info,caretaker,date,dbh,plot_size,latitude,longitude,decade,geometry,index_right,name,cartodb_id,created_at,updated_at,decade_group
0,30460,DPW Maintained,Victorian Box,470 Union St,1.0,Sidewalk: Curb side : Cutout,Private,1956-05-11,19.0,4x4,37.800739,-122.407264,1950s,POINT (-122.40726 37.80074),10,North Beach,37,2013-02-10 05:44:04.653000+00:00,2013-02-10 05:44:04.923000+00:00,1950s-1960s
1,30454,DPW Maintained,Victorian Box,381 Union St,1.0,Sidewalk: Curb side : Cutout,Private,1956-05-11,8.0,3x3,37.800813,-122.405701,1950s,POINT (-122.40570 37.80081),10,North Beach,37,2013-02-10 05:44:04.653000+00:00,2013-02-10 05:44:04.923000+00:00,1950s-1960s
2,30428,DPW Maintained,Victorian Box,434 Union St,1.0,Sidewalk: Curb side : Cutout,Private,1956-05-11,13.0,7x3,37.800822,-122.406603,1950s,POINT (-122.40660 37.80082),10,North Beach,37,2013-02-10 05:44:04.653000+00:00,2013-02-10 05:44:04.923000+00:00,1950s-1960s
56,57595,DPW Maintained,Sycamore: London Plane,1300 Battery St,1.0,Sidewalk: Curb side : Cutout,Private,1976-11-05,13.0,3x3,37.803355,-122.401915,1970s,POINT (-122.40191 37.80336),10,North Beach,37,2013-02-10 05:44:04.653000+00:00,2013-02-10 05:44:04.923000+00:00,1970s-1980s
174,4703,DPW Maintained,Brisbane Box,930 Columbus Ave,1.0,Sidewalk: Curb side : Cutout,DPW,1984-03-19,36.0,5x6,37.803071,-122.414010,1980s,POINT (-122.41401 37.80307),10,North Beach,37,2013-02-10 05:44:04.653000+00:00,2013-02-10 05:44:04.923000+00:00,1970s-1980s
449,21774,DPW Maintained,Southern Magnolia,2495X Taylor St,7.0,Sidewalk: Curb side : Cutout,DPW,1990-03-30,6.0,4x4,37.805168,-122.415279,1990s,POINT (-122.41528 37.80517),10,North Beach,37,2013-02-10 05:44:04.653000+00:00,2013-02-10 05:44:04.923000+00:00,1990s-2000s
450,21770,DPW Maintained,Southern Magnolia,2495X Taylor St,3.0,Sidewalk: Curb side : Cutout,DPW,1990-03-30,12.0,3X3,37.805168,-122.415279,1990s,POINT (-122.41528 37.80517),10,North Beach,37,2013-02-10 05:44:04.653000+00:00,2013-02-10 05:44:04.923000+00:00,1990s-2000s
452,21767,DPW Maintained,Southern Magnolia,2400X Taylor St,10.0,Sidewalk: Curb side : Cutout,DPW,1990-03-30,12.0,3X3,37.804514,-122.414971,1990s,POINT (-122.41497 37.80451),10,North Beach,37,2013-02-10 05:44:04.653000+00:00,2013-02-10 05:44:04.923000+00:00,1990s-2000s
453,21766,DPW Maintained,Southern Magnolia,2400X Taylor St,9.0,Sidewalk: Curb side : Cutout,DPW,1990-04-01,12.0,3X3,37.804514,-122.414971,1990s,POINT (-122.41497 37.80451),10,North Beach,37,2013-02-10 05:44:04.653000+00:00,2013-02-10 05:44:04.923000+00:00,1990s-2000s
454,21760,DPW Maintained,Southern Magnolia,2400X Taylor St,3.0,Sidewalk: Curb side : Cutout,DPW,1990-04-01,12.0,3X3,37.804514,-122.414971,1990s,POINT (-122.41497 37.80451),10,North Beach,37,2013-02-10 05:44:04.653000+00:00,2013-02-10 05:44:04.923000+00:00,1990s-2000s


In [16]:
# Selecting only the desired columns
columns_to_keep = ['species', 'date', 'latitude', 'longitude', 'decade', 'geometry', 'name','decade_group']
df_selected = df_joined[columns_to_keep]

# Saving the selected data to a new CSV file
df_selected.to_csv('selected_data.csv', index=False)

In [25]:
df_selected['species'].nunique()

16

In [27]:
unique_decade_groups = df_joined['decade_group'].unique()
unique_decade_groups


array(['1950s-1960s', '1970s-1980s', '1990s-2000s', '2010s-2020s'],
      dtype=object)